# Project: Product Management From Scratch

The boutique ice cream sandwich startup you've joined, Snackr, has gotten by on the personality of it's founder, I. Sandwich (real name?), for too long.  The last 2 quarters of features the team has launched to the website have not landed with customers.

You've decided it's time to step up, lay out a plan for how you're going to identify paths forward which are more likely to succeed than "whatever I. Sandich happened to promise a random person he met at a bar last night".  You pitch the team on an iterative design process which brings qualitative and quantitative techniques together in order to minimize industry standard risk types so that you can move forward confidently.

You have some random user event logs you can dig through in order to surface user personas and a beginning journey.  You tell the team you'd like to collect all of the known assumptions before you launch the next experiment to the site so you have the best chance of success.  You tell the CEO and Board (wait - is it really just his friends from college and mom?) that at the end of your week long design sprint you're going to launch an experiment.  

After that you'll come back to them to share what you've learned and present some ideas for a paths forward.  

<button id="ulab-button-0c1c8b8a" class="ulab-btn--primary"></button>

<!--
%%ulab_page_divider
--><hr/>

In [19]:
import numpy as np
import pandas as pd
import glob
import datetime

In [3]:
log_files = glob.glob(r'events_*.csv')
df = pd.concat((pd.read_csv(f) for f in log_files))

## Section 1 - User Journey

In [11]:
#S1
df.describe()

,event_id,user_uuid,event_time,device_type,session_uuid,experiment_group,user_country,event_page,event_type
count,1247914,1247914,1247914,1247914,1247914,1247914,1247914,1247914,1247914
unique,1247914,81641,525817,4,227081,2,5,4,5
top,bff56d56-b039-44e8-8bb5-9a0e99a83f2c,9d3592ba-f1ed-45ce-a7c2-7f242ee353a9,2019-09-27 16:47:06,mobile_web,68ef8208-8aa6-44e6-a137-d1ae72afa22c,experiment,CN,home_page,view
freq,1,49,12,312812,10,624849,250957,482690,817324


In [12]:
#S1_1
df.device_type.unique()

array(['desktop_web', 'ios', 'mobile_web', 'android'], dtype=object)

In [14]:
#S1_1
df.user_country.unique()

array(['JP', 'FR', 'CN', 'US', 'UK'], dtype=object)

In [15]:
#S1_2
df.event_page.unique()

array(['home_page', 'search_page', 'item_page', 'marketing_page'], dtype=object)

In [16]:
#S1_2
df.event_type.unique()

array(['view', 'share', 'favorite', 'cart', 'purchase'], dtype=object)

In [17]:
#S1_3
#TODO - MATT - Generate opportunities in dataset.
#1 - bounce
#2 - bug
#3 - typo
#4 - l10n
#5 - price
#6 - flow

## Section 2 - Forumulate Hypotheses

Because **evidence**   
we believe that by **product change**  
for **defined audience**  
users will be **XX% (more/less)** likely   
to **(view,share,favorite,cart,purchase)**  
  
In order to be certain we will run an a/b test for **DD days**  

## Section 3 - Create Wireframes of Change

TKTK - Is this a standout section?

## Section 4 - Run Experiment

This is the data for the course.  Maybe a fun gif here of the seasons changing.  Can we get rights to something like this?

![time passes through the seasons as we wait for the experiment to finish running](seasons.gif)


## Section 5 - Draw Product Insights


In [21]:
#S5_1
df.describe()

,event_id,user_uuid,event_time,device_type,session_uuid,experiment_group,user_country,event_page,event_type
count,1247914,1247914,1247914,1247914,1247914,1247914,1247914,1247914,1247914
unique,1247914,81641,525817,4,227081,2,5,4,5
top,bff56d56-b039-44e8-8bb5-9a0e99a83f2c,9d3592ba-f1ed-45ce-a7c2-7f242ee353a9,2019-09-27 16:47:06,mobile_web,68ef8208-8aa6-44e6-a137-d1ae72afa22c,experiment,CN,home_page,view
freq,1,49,12,312812,10,624849,250957,482690,817324


In [22]:
#S5_2
desired_p = 0.5
z_score = 1.960  #TODO Student to get the z-score for 95% confidence range

total_control = 10944 #TODO Student to get the total number of events
total_experiment = 11173 #TODO Student to get the total number of events


standard_deviation = np.sqrt((0.5*0.5)/(total_control + total_experiment))
margin_error = standard_deviation * z_score

confidence_interval = (desired_p-margin_error, desired_p+margin_error)

p_hat = total_control / (total_control + total_experiment)

print("p_hat:\t\t", p_hat)
print("confidence int:\t", confidence_interval)
print("Continue analysis?:", p_hat > confidence_interval[0] and p_hat < confidence_interval[1])

p_hat:		 0.4948229868427002
confidence int:	 (0.49341034061600114, 0.50658965938399891)
Continue analysis?: True


In [ ]:
#S5_3
tktk - out of scope?

In [25]:
#S5_4
#tktk - to do

In [26]:
#S5_5
#tktk - todo

## Section 6

#S6_1 tktk - todo
#S6_2 tktk - todo
#S6_3 tktk - todo
#S6_4 tktk - todo